In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/chest_xray"
base_url = "/content/gdrive/My Drive/chest_xray"

In [ ]:
%tensorflow_version 1.x
import numpy as np
import os
import shutil
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD, Adam
import numpy as np
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image
from skimage import io, color
from keras.models import load_model
from keras.applications.mobilenet import MobileNet,preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

In [ ]:
epoch = 100
train_dir = base_url + '/train'
validation_dir = base_url + '/val'
test_dir = base_url + '/test'
training_count = 5216
testing_count = 16

In [ ]:
model = Sequential()
model.add(MobileNet(include_top=False, pooling='avg',))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(optimizer="adam",
            loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function= preprocess_input,
    rescale=1./255,
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=  preprocess_input,
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224),
                                                batch_size=32, class_mode='categorical',
                                                )

validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical',
                                                )

history = model.fit_generator(train_generator, epochs=epoch, steps_per_epoch=training_count // 32,
                            validation_data=validation_generator, validation_steps=testing_count // 32)


In [ ]:
datagen = ImageDataGenerator(rescale=1./255,preprocessing_function = preprocess_input)

generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical',
    shuffle=False,
)

generator.reset()

In [ ]:
predict = model.predict_generator(generator, 624) #melakukan prediksi dari dataset testing (624 adalah jumlah data testing)

predicted = np.argmax(predict, axis=1) # mengambil list class yang diprediksi
actual = generator.classes[generator.index_array] #mengambil list class yang sebenarnya (ground truth)

test_accuracy = accuracy_score(actual, predicted) #menghitung akurasi model dengan membandingkan hasil prediksi dengan ground truth

total_actual = actual
total_predicted = predicted

val_loss, val_accuracy = model.evaluate_generator(evaluate_generator, 624 // 32) #mengevaluasi model menggunakan evaluate generator terhadap dataset testing

val_training = history.history['acc'][-1]
val_loss_training = history.history['loss'][-1]
print('val acc : '+ str(val_accuracy)) 
print('val loss : ' + str(val_loss))
print('training acc :' + str(val_training))
print('training loss :' + str(val_loss_training))
print('test accuracy :' + str(test_accuracy))

print(classification_report(total_actual, total_predicted)) #menampilkan metrices precission,recall,f1score,dan akurasi model dengan menggunakan data perbandingan prediksi dan ground truth
print(confusion_matrix(total_actual, total_predicted)) #menampilkan confusion matrix untuk melihat performa klasifikasi pada tiap class

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

epochs = range(epoch)

plt.plot(epochs, acc, 'r', label='training')
plt.plot(epochs, val_acc, 'b', label='testing')
plt.legend()
plt.show()

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, 'r', label='training')
plt.plot(epochs, val_loss, 'b', label='testing')
plt.legend()
plt.show()